<a href="https://colab.research.google.com/github/ag4267research1/Hybrid-Quantum-PDE-constrained-Optimization/blob/main/PDE_Quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We start from the simple 1-D Poisson equation

$$
-u^{\prime \prime}(x)=1, \quad x \in(0,1), \quad u(0)=u(1)=0 .
$$

- $u(x)$ is the state (solution of the PDE).
- The right-hand side 1 acts like a source term.

We discretize the interval $[0,1]$ with 3 equal subintervals:
- Grid points: $x_0=0, x_1=1 / 3, x_2=2 / 3, x_3=1$.
- Grid spacing: $h=1 / 3$.
- Unknowns: values at interior points $u_1 \approx u\left(x_1\right), u_2 \approx u\left(x_2\right)$.

Second derivative with central differences:

$$
u^{\prime \prime}\left(x_i\right) \approx \frac{u_{i-1}-2 u_i+u_{i+1}}{h^2} .
$$


Our PDE is $-u^{\prime \prime}(x)=1$. So at each interior node $x_i$ :

$$
-\frac{u_{i-1}-2 u_i+u_{i+1}}{h^2}=1 .
$$


Use the boundary conditions $u_0=0, u_3=0$ :
- At $x_1$ :

$$
-\frac{0-2 u_1+u_2}{h^2}=1 \Rightarrow 2 u_1-u_2=h^2
$$

- At $x_2$ :

$$
-\frac{u_1-2 u_2+0}{h^2}=1 \Rightarrow-u_1+2 u_2=h^2
$$


So in matrix form:

$$
A u=b
$$

with

$$
A=\left(\begin{array}{cc}
2 & -1 \\
-1 & 2
\end{array}\right), \quad u=\binom{u_1}{u_2}, \quad b=h^2\binom{1}{1} .
$$

## Classical Solve

In [ ]:
import numpy as np
import time


In [ ]:

# Matrix A and RHS b from the PDE
h = 1/3
A = np.array([[2, -1],
              [-1, 2]], dtype=float)
b = (h**2) * np.array([1.0, 1.0])

# Classical solve and timing
t0 = time.time()
u_classical = np.linalg.solve(A, b)
t1 = time.time()

print("Classical solution u =", u_classical)
print("Classical solve time (s) =", t1 - t0)


Classical solution u = [0.11111111 0.11111111]
Classical solve time (s) = 0.0004417896270751953


## Hybrid Algorithm

In [ ]:
!pip install qiskit
!pip install qiskit qiskit-aer
!pip install qiskit[visualization]
!pip install pylatexenc


In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister
from qiskit.circuit.library import Initialize
from qiskit_aer import Aer
from qiskit import transpile
from scipy.linalg import expm
from qiskit.circuit.library import UnitaryGate

This gives a 2-component vector that we interpret as amplitudes of a 1-qubit state:

$$
|b\rangle=\beta_0|0\rangle+\beta_1|1\rangle, \quad \beta_i=b_i /\|b\| .
$$


We use Qiskit's Initialize to create this state:

In [ ]:
# Normalize b as state |b>
b_normalized = b / np.linalg.norm(b)

init_b = Initialize(b_normalized)
qc_prep = QuantumCircuit(1)  # 1 system qubit
qc_prep.append(init_b, [0])

print("State-prep circuit for |b>:")
print(qc_prep.draw('text'))


State-prep circuit for |b>:
   ┌─────────────────────────────┐
q: ┤ Initialize(0.70711,0.70711) ├
   └─────────────────────────────┘


QLSA works by running Quantum Phase Estimation (QPE) on a unitary whose eigenvalues contain the eigenvalues $\lambda$ of the matrix $A$.

For a $2 \times 2$ Hermitian A we can exponentiate it directly:

In [ ]:
# Scale A for numerical stability; we only need some unitary e^{iAt}
t_param = 1.0
A_scaled = A / np.linalg.norm(A)   # keep eigenvalues in [-1,1]
U = expm(1j * A_scaled * t_param)  # 2x2 unitary

U_gate = UnitaryGate(U, label="e^{iA}")

This U_gate approximates $e^{i A t}$. In real HHL, we'd use controlled powers of this unitary inside QPE.


In [ ]:
ampl_d, n_qubits_d, norm_d = to_normalized_amplitudes(d)
assert n_qubits_d == n_qubits  # both should be 6

init_d = Initialize(ampl_d)
qc_d = QuantumCircuit(n_qubits)
qc_d.append(init_d, range(n_qubits))

print("\nCircuit 2: state preparation for |d> (direction)")
print(qc_d.draw("text"))



Circuit 2: state preparation for |d> (direction)
     »
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
q_4: »
     »
q_5: »
     »
«     ┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                                                                                                                                                                                                                                                                                             



We use 3 qubits:
- qubit 0: phase ancilla for QPE,
- qubit 1: system qubit (stores $|b\rangle$ and then $|u\rangle$ ),
- qubit 2: rotation ancilla for the $1 / \lambda$ step.

In [ ]:
def build_QSLA_circuit(b_state, U_gate):
    """
    Tiny QSLA circuit for 2x2 A using:
    - 1 system qubit
    - 1 phase ancilla (QPE)
    - 1 rotation ancilla
    """
    qc = QuantumCircuit(3, 1)  # [phase, system, rot], 1 classical bit

    q_phase = 0
    q_sys   = 1
    q_rot   = 2

    # 1. Prepare |b> on system qubit
    qc.append(Initialize(b_state), [q_sys])

    # 2. Put phase ancilla into superposition (start QPE)
    qc.h(q_phase)

    # 3. Controlled-U from phase onto system
    qc.append(U_gate.control(1), [q_phase, q_sys])

    print("\nCircuit after controlled time evolution (QPE part):")
    print(qc.draw('text'))

    # 4. (With 1 phase qubit, inverse QFT is just another H)
    qc.h(q_phase)

    print("\nCircuit after inverse QFT on phase ancilla:")
    print(qc.draw('text'))

    # 5. Controlled rotation on q_rot, conditioned on q_phase
    #    In true HHL, angle depends on eigenvalue; here we choose some angle theta.
    theta = np.pi / 3  # arbitrary example, just to see structure
    qc.cry(theta, q_phase, q_rot)

    print("\nCircuit after controlled rotation encoding ~1/lambda:")
    print(qc.draw('text'))

    # 6. Uncompute phase ancilla (reverse of steps 2-3)
    qc.h(q_phase)
    qc.append(U_gate.control(1).inverse(), [q_phase, q_sys])
    qc.h(q_phase)

    print("\nCircuit after uncomputing phase ancilla (inverse QPE):")
    print(qc.draw('text'))

    # 7. Measure rotation ancilla (success flag)
    qc.measure(q_rot, 0)

    print("\nFinal QLSA circuit:")
    print(qc.draw('text'))

    return qc

qc_hhl = build_QSLA_circuit(b_normalized, U_gate)





Circuit after controlled time evolution (QPE part):
                  ┌───┐                       
q_0: ─────────────┤ H ├─────────────────■─────
     ┌────────────┴───┴────────────┐┌───┴────┐
q_1: ┤ Initialize(0.70711,0.70711) ├┤ e^{iA} ├
     └─────────────────────────────┘└────────┘
q_2: ─────────────────────────────────────────
                                              
c: 1/═════════════════════════════════════════
                                              

Circuit after inverse QFT on phase ancilla:
                  ┌───┐                       ┌───┐
q_0: ─────────────┤ H ├─────────────────■─────┤ H ├
     ┌────────────┴───┴────────────┐┌───┴────┐└───┘
q_1: ┤ Initialize(0.70711,0.70711) ├┤ e^{iA} ├─────
     └─────────────────────────────┘└────────┘     
q_2: ──────────────────────────────────────────────
                                                   
c: 1/══════════════════════════════════════════════
                                                   

Circuit af

Conceptually:
1. Start in $|0\rangle_{\text {phase }} \otimes|b\rangle_{\text {sys }} \otimes|0\rangle_{\text {rot }}$.
2. After QPE + inverse QFT, phase ancilla "contains" eigenvalue information of A.
3. The controlled rotation applies an amplitude factor that mimics multiplying by $1 / \lambda$.
4. After uncomputing the phase ancilla, the system qubit is in a state proportional to $A^{-1} b$, i.e. the solution vector $u$ up to normalization.
5. Measuring rotation ancilla = 1 would correspond to a "successful" run in full QLSA; we ignore postselection here and look at the pure state.

This is a simplified QLSA, but structurally it's the real algorithm: QPE → controlled $1 / \lambda$ rotation → inverse QPE.

We want the statevector of the circuit before measurement. Qiskit will not give you a statevector if measurements are present, so we must remove them and explicitly save the state.

In [ ]:
backend_sv = Aer.get_backend("aer_simulator")

qc_no_meas = qc_hhl.copy()
qc_no_meas.remove_final_measurements()  # drop measure(q_rot,0)
qc_no_meas.save_statevector()           # tell simulator to store |ψ>

compiled = transpile(qc_no_meas, backend_sv)
result = backend_sv.run(compiled).result()
state = result.data(0)["statevector"]

print("Final 3-qubit statevector (phase, sys, rot):")
print(state)


Final 3-qubit statevector (phase, sys, rot):
Statevector([-0.52053727-0.47510517j,  0.00993035-0.01087994j,
             -0.52053727-0.47510517j,  0.00993035-0.01087994j,
             -0.00647418-0.00590912j, -0.03706058+0.04060451j,
             -0.00647418-0.00590912j, -0.03706058+0.04060451j],
            dims=(2, 2, 2))


The basis ordering is $\left|q_0 q_1 q_2\right\rangle=\mid$ phase, sys, rot $\rangle$.
To interpret the system qubit as our solution, we look at entries where rot=1 and project onto the system qubit. For a rough demo, we can just extract the subsystem amplitudes and renormalize:

In [ ]:
def swap_test_circuit(state_a, state_b):
    """
    Build SWAP test for two 1-qubit states |a>, |b>.
    state_a, state_b: 2-element normalized complex vectors.
    """
    from qiskit import QuantumCircuit

    qc = QuantumCircuit(3, 1)  # anc, a, b; measure anc

    # Prepare |a> on qubit 1, |b> on qubit 2
    qc.append(Initialize(state_a), [1])
    qc.append(Initialize(state_b), [2])

    # SWAP test
    qc.h(0)
    qc.cswap(0, 1, 2)
    qc.h(0)
    qc.measure(0, 0)

    print("\nSWAP test circuit:")
    print(qc.draw('text'))

    return qc


In [ ]:
u_quantum = np.array([state[2], state[3]])   # basis |010>, |011>
u_quantum = u_quantum / np.linalg.norm(u_quantum)

print("\nQuantum-estimated u (directional):")
print(u_quantum)

u1_norm = u_classical / np.linalg.norm(u_classical)
u2_norm = u_quantum

init_u1 = Initialize(u1_norm)
init_u2 = Initialize(u2_norm)

qc_test = QuantumCircuit(3, 1)
qc_test.append(init_u1, [0])
qc_test.append(init_u2, [1])

qc_swap = swap_test_circuit(u1_norm, u2_norm)
qc_test.compose(qc_swap, inplace=True)

print("\nSwap test circuit:")
print(qc_test.draw('text'))


Quantum-estimated u (directional):
[-0.73844289-0.67399216j  0.01408736-0.01543447j]

SWAP test circuit:
                            ┌───┐                           ┌───┐┌─┐
q_0: ───────────────────────┤ H ├─────────────────────────■─┤ H ├┤M├
               ┌────────────┴───┴────────────┐            │ └───┘└╥┘
q_1: ──────────┤ Initialize(0.70711,0.70711) ├────────────X───────╫─
     ┌─────────┴─────────────────────────────┴──────────┐ │       ║ 
q_2: ┤ Initialize(-0.73844-0.67399j,0.014087-0.015434j) ├─X───────╫─
     └──────────────────────────────────────────────────┘         ║ 
c: 1/═════════════════════════════════════════════════════════════╩═
                                                                  0 

Swap test circuit:
               ┌─────────────────────────────┐           »
q_0: ──────────┤ Initialize(0.70711,0.70711) ├───────────»
     ┌─────────┴─────────────────────────────┴──────────┐»
q_1: ┤ Initialize(-0.73844-0.67399j,0.014087-0.015434j) ├»
     ├───────────

In [ ]:
# Classical timing already computed earlier

t0 = time.time()
_ = backend_sv.run(compiled).result()
t1 = time.time()

print("QLSA run time (s) =", t1 - t0)


QLSA run time (s) = 0.027222156524658203
